# Classification de texte avec LSTM sous PyTorch

Ce notebook détaille l'entraînement et la prédiction d'un modèle LSTM (Long Short-Term Memory) pour la classification de texte. 
Chaque étape est expliquée, du chargement des données séquentielles à la génération du fichier de soumission.

---

## Concepts clés

- **LSTM** : Réseau de neurones récurrent (RNN) conçu pour traiter des séquences et mémoriser des dépendances à long terme. Idéal pour le texte.
- **Embedding** : Transformation des indices de mots en vecteurs denses, apprises pendant l'entraînement.
- **Pondération des classes** : Correction du déséquilibre des classes en pondérant la perte.
- **CrossEntropyLoss** : Fonction de perte adaptée à la classification multi-classes.
- **Adam** : Optimiseur efficace pour l'entraînement des réseaux de neurones.

---

In [ ]:
# Importation des bibliothèques nécessaires
import numpy as np
import joblib
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd

## Version et introduction

On affiche la version du script et on explique le but : entraîner un modèle LSTM pour la classification de texte séquentiel.

In [ ]:
print("V2_1.2.1 – Entraînement + Prédiction LSTM PyTorch")

## 1. Chargement des artefacts séquentiels

On charge les matrices numpy contenant les séquences d'indices de mots (`X_train_seq`, `X_test_seq`), les labels, le vocabulaire (mapping mot→indice) et l'encodeur de labels.

- `X_train_seq` et `X_test_seq` : matrices (n_samples, max_len) où chaque ligne est une séquence d'indices de mots.
- `stoi` : dictionnaire mot→indice, utilisé pour l'embedding.
- `le` : encodeur pour transformer les labels en entiers et inversement.

In [ ]:
X_train = np.load('X_train_seq.npy')
y_train = np.load('y_train.npy')
X_test  = np.load('X_test_seq.npy')
stoi     = joblib.load('vocab_stoi.pkl')
le       = joblib.load('label_encoder.pkl')

## 2. Conversion en tensors PyTorch

On convertit les matrices numpy en tensors PyTorch pour pouvoir les utiliser dans le DataLoader et le modèle. 
On crée un DataLoader pour itérer sur les données par mini-batchs lors de l'entraînement.

In [ ]:
X_tr = torch.tensor(X_train, dtype=torch.long)
y_tr = torch.tensor(y_train, dtype=torch.long)
X_te = torch.tensor(X_test,  dtype=torch.long)

train_ds = TensorDataset(X_tr, y_tr)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

## 3. Définition du modèle LSTM

Le modèle comprend :
- Une couche d'embedding (transforme chaque indice de mot en vecteur dense)
- Un LSTM (traite la séquence de vecteurs et capture les dépendances contextuelles)
- Un dropout (régularisation)
- Une couche linéaire finale pour la classification

Mathématiquement, le LSTM apprend à mémoriser les informations pertinentes dans la séquence grâce à ses cellules mémoire et ses portes (input, forget, output). La sortie finale (dernier état caché) est utilisée pour la classification.

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_classes, pad_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)
        self.lstm      = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.dropout   = nn.Dropout(0.3)
        self.fc        = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        emb, _ = self.lstm(self.embedding(x))
        last = emb[:, -1, :]
        return self.fc(self.dropout(last))

vocab_size  = len(stoi)
embed_dim   = 128
hidden_dim  = 64
num_classes = len(le.classes_)

model = LSTMClassifier(vocab_size, embed_dim, hidden_dim, num_classes, pad_idx=stoi['<PAD>'])

## 4. Configuration de l’entraînement et pondération des classes

Pour corriger le déséquilibre des classes, on calcule des poids inverses à la fréquence de chaque classe et on les passe à la fonction de perte `CrossEntropyLoss`.

- **CrossEntropyLoss** : combine softmax et log-vraisemblance pour la classification multi-classes.
- **Adam** : optimiseur adaptatif efficace pour l'entraînement des réseaux de neurones.

In [ ]:
# Calcul manuel des poids de classes
counts = np.bincount(y_train, minlength=num_classes)
total  = y_train.shape[0]
counts = np.where(counts == 0, 1, counts)  # éviter division par zéro
class_weights = torch.tensor((total / counts), dtype=torch.float)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs    = 15

## 5. Entraînement du modèle

À chaque époque, pour chaque batch :
- On calcule les logits (sorties non normalisées du modèle)
- On calcule la perte pondérée
- On rétro-propage le gradient et on met à jour les poids
- On mesure la précision sur le batch

La perte et la précision moyennes sont affichées à chaque époque pour suivre la convergence.

In [ ]:
for epoch in range(1, epochs+1):
    model.train()
    total_loss = 0.0
    correct    = 0
    total      = 0

    for xb, yb in train_loader:
        optimizer.zero_grad()
        logits = model(xb)
        loss   = criterion(logits, yb)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()
        total   += yb.size(0)

    train_loss = total_loss / len(train_loader)
    train_acc  = correct / total
    print(f"Epoch {epoch}/{epochs} — loss: {train_loss:.4f} — acc: {train_acc:.4f}")

## 6. Prédiction sur le test

On met le modèle en mode évaluation, on calcule les logits pour chaque séquence de test, puis on prend la classe avec la probabilité la plus élevée (`argmax`).
On utilise le label encoder pour retrouver les noms de catégories d'origine.

In [ ]:
model.eval()
with torch.no_grad():
    logits = model(X_te)
    preds  = torch.argmax(logits, dim=1).cpu().numpy()
    cats   = le.inverse_transform(preds)

## 7. Génération du fichier de soumission

On prépare le fichier de soumission au format attendu, associant chaque Id de test à la catégorie prédite.

In [ ]:
print("Saving submission2_3.csv...")
test_ids   = pd.read_json('test_mini.json', orient='records')['Id']
submission = pd.DataFrame({'Id': test_ids, 'Category': cats})
submission.to_csv('submission2_3.csv', index=True)
print("✅ submission2_3.csv généré.")

---

## Concepts mathématiques et conclusion

- **Embedding** : chaque mot est représenté par un vecteur dense appris pendant l'entraînement. L'embedding permet de capturer la similarité sémantique entre mots.
- **LSTM** : pour chaque séquence $x_1, ..., x_T$, le LSTM produit une séquence d'états cachés $h_t$ qui résument l'information du passé. Le dernier état caché est utilisé pour la classification.
- **CrossEntropyLoss** : pour chaque exemple, la perte est $-\log(p_{y})$ où $p_{y}$ est la probabilité prédite pour la vraie classe.

Ce pipeline montre comment passer de séquences de mots à des prédictions de classes avec un réseau LSTM, en tenant compte du contexte et de l'ordre des mots dans le texte.